# Least Squares Tutorial

This notebook walks through the theory and implementation of the **Least Squares** estimation problem, as introduced in the `pykal` documentation. It covers:

1. Motivation from inverse problems
2. Batch linear least squares
3. Sequential (recursive) least squares
4. Sensor calibration example
5. Connections to Kalman Filtering

For derivations and full context, refer to the Theory & Background section in the documentation.

## Motivation: Inverse Problems

We are given a system where outputs `y` are related to some unknown input `x` via a (possibly nonlinear) function `f(x)`:

$$ f(x) = y $$

But often this equation has no solution because `y` is not in the image of `f`. So instead, we solve the least-squares problem:

$$ x^* = \arg\min_x \|f(x) - y\|_2^2 $$

## Batch Linear Least Squares

Assume a linear model:

$$ y = A x + \varepsilon $$

The solution is given by the normal equations:

$$ x^* = (A^T A)^{-1} A^T y $$

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Simulate sensor calibration data
np.random.seed(42)
T = np.linspace(0, 100, 20)  # known temperatures
true_params = np.array([0.1, 0.05])  # offset and sensitivity
y = true_params[0] + true_params[1] * T + np.random.normal(0, 0.1, size=T.shape)

# Design matrix A
A = np.vstack([np.ones_like(T), T]).T
# Batch least squares
x_star = np.linalg.inv(A.T @ A) @ A.T @ y
x_star

array([0.17745666, 0.04810827])

In [ ]:
# Plot results
plt.scatter(T, y, label='Noisy measurements')
plt.plot(T, A @ x_star, label='Least-squares fit', color='red')
plt.xlabel('Temperature (°C)')
plt.ylabel('Voltage (V)')
plt.title('Sensor Calibration via Batch Least Squares')
plt.legend()
plt.grid(True)
plt.show()

## Sequential (Recursive) Least Squares

Update the estimate with each new measurement using:

$$
K_k = P_{k-1} a_k (a_k^T P_{k-1} a_k + \sigma^2)^{-1},
\quad
x_k = x_{k-1} + K_k (y_k - a_k^T x_{k-1}),
\quad
P_k = (I - K_k a_k^T) P_{k-1}
$$

In [ ]:
# Recursive Least Squares (RLS) implementation
x_rls = np.zeros(2)
P = 100 * np.eye(2)
sigma2 = 0.01

for i in range(len(T)):
    a_k = A[i].reshape(2, 1)
    y_k = y[i]
    K = P @ a_k / (a_k.T @ P @ a_k + sigma2)
    x_rls = x_rls + (K.flatten() * (y_k - a_k.T @ x_rls))
    P = (np.eye(2) - K @ a_k.T) @ P

x_rls

This estimate matches the batch solution closely, and is suitable for real-time updates.

In practice, this approach is extended to dynamic systems via the **Kalman Filter**, which we treat in a separate notebook.